<a href="https://colab.research.google.com/github/rashmikb26/BoomBike/blob/main/notebook_cement_sales_and_demand_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Cement Sales & Demand Prediction 89

In [20]:
#importing important Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


In [22]:
#loading the data and creating column 't'
import os

#1) From Github (Files < 25MB)

#filename = "https://github.com/rashmikb26/CementSalesForecasting/blob/main/89cement%20data%20.csv"
# provide the raw data url
filename =" https://raw.githubusercontent.com/rashmikb26/CementSalesForecasting/main/89cement%20data%20.csv?token=GHSAT0AAAAAACSFEYOLCA23IP4Z3BC4BJ7WZSCJKTQ"
url = filename
df1 = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

cement = pd.read_csv(filename)
cement = cement.iloc[:155, :8]



HTTPError: HTTP Error 404: Not Found

In [ ]:
cement['t'] =np.arange(1,156)
cement.tail(10)

In [ ]:
# Interchanging position of feature t
cement = cement.iloc[:, [0,8,1,2,3,4,5,6,7]]
cement.head(10)

# AutoEDA

In [ ]:
# performing autoEDA
!pip install pandas-profiling
!pip install pydantic-settings

from pydantic_settings import BaseSettings
import pandas-profiling

autoeda = ProfileReport(cement, explorative= True)
autoeda

# Correlation matrix

In [ ]:
# checking correlation between each features

cement.corr()

# Pairplot

In [ ]:
sns.pairplot(cement)

In [ ]:
cement.columns

# Time Plots

In [ ]:
# time plot is basically a line plot showing the evolution of the time series over time.

sns.lineplot('t', 'Sales ',data= cement)


In [ ]:
sns.lineplot('t', 'Production',data= cement)

In [ ]:
sns.lineplot('t', 'demand',data= cement)

In [ ]:
sns.lineplot('t', 'population',data= cement)

In [ ]:
sns.lineplot('t', 'gdp',data= cement)

In [ ]:
sns.lineplot('t', 'disbusment',data= cement)

In [ ]:
sns.lineplot('t', 'interestrate', data= cement)

In [ ]:
cement.columns

In [ ]:
# checking outliers

sns.boxplot(cement['Sales '])


In [ ]:
#!pip install feature_engine

In [ ]:
#outliers are present in this feature so need to treat them
#lets use Winsorization

from feature_engine.outliers import Winsorizer
winsor = Winsorizer(capping_method='iqr',
                   fold= 1.5,
                   tail= 'both',
                   variables=['Sales '])
cement['Sales '] = winsor.fit_transform(cement[['Sales ']])
sns.boxplot(cement['Sales '])

In [ ]:
sns.boxplot(cement['demand'])

In [ ]:
sns.boxplot(cement['Production'])

In [ ]:
sns.boxplot(cement['population'])

In [ ]:
sns.boxplot(cement['gdp'])

In [ ]:
from feature_engine.outliers import Winsorizer
winsor = Winsorizer(capping_method='iqr',
                   fold= 1.5,
                   tail= 'both',
                   variables=['gdp'])
cement['gdp'] = winsor.fit_transform(cement[['gdp']])
sns.boxplot(cement['gdp'])

In [ ]:
sns.boxplot(cement['disbusment'])

In [ ]:
sns.boxplot(cement['interestrate'])

In [ ]:
sns.lineplot('t','gdp', data=cement)

# Forecasting for Sales

# Linear Model

In [ ]:
# breaking up a time series into components, most notably: a trend component, a seasonal component, and a residual component

from statsmodels.tsa.seasonal import seasonal_decompose

decompose_ts_add = seasonal_decompose(cement['Sales '], model = "additive", period = 4)
print(decompose_ts_add.trend)
print(decompose_ts_add.seasonal)
print(decompose_ts_add.resid)
print(decompose_ts_add.observed)
decompose_ts_add.plot()

In [ ]:
decompose_ts_mul = seasonal_decompose(cement['Sales '], model = "multiplicative", period = 4)
decompose_ts_mul.plot()

In [ ]:
# filtering requied features for sales forecasting

cementsales = cement.loc[:, ['Month','Sales ', 't']]
cementsales.head()

In [ ]:
# squaring t value as it will further requied for model

cementsales['tsquare'] = cementsales['t'] * cementsales['t']
cementsales.head()
cementsales.rename(columns = {'Sales ' : 'sales'}, inplace = True)

In [ ]:
# taking the log of sales values as it will be required for the exponential model

cementsales['log_cement'] = np.log(cementsales['sales'])

cementsales.head()

In [ ]:
p = cementsales["Month"][0]
p[0:3]



In [ ]:
cementsales['months']= 0

for i in range(155):
    p = cementsales["Month"][i]
    cementsales['months'][i]= p[0:3]

cementsales

In [ ]:
# converting months catagorical features to numeric columns by using pandas get_dummies function

month_dummies = pd.DataFrame(pd.get_dummies(cementsales['months']))
month_dummies.head()

In [ ]:
# concating data

cementsales1 = pd.concat([cementsales, month_dummies], axis= 1)
cementsales1.head()

In [ ]:
cementsales1.shape

In [ ]:
# splitting data into train & test

train = cementsales1.head(143)
test = cementsales1.tail(12)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
test.tail()

# Linear Model

In [ ]:
#importing linear Model, error matrix for error calculation
#checking different models for better accuracy

import statsmodels.formula.api as api
from sklearn.metrics import mean_absolute_percentage_error

#Linear Model

linear_model = api.ols('sales ~ t', data = train).fit()
pred_linear =  pd.Series(linear_model.predict(pd.DataFrame(test['t'])))
rmse_linear = np.sqrt(np.mean((np.array(test['sales']) - np.array(pred_linear))**2))
mapa_linear = mean_absolute_percentage_error(test['sales'], pd.DataFrame(pred_linear))

print('mapa : ', mapa_linear)
print('rmse : ',rmse_linear)

In [ ]:
# Exponential Model

Exp = api.ols('log_cement ~ t', data = train).fit()
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(test['t'])))
rmse_Exp = np.sqrt(np.mean((np.array(test['sales']) - np.array(np.exp(pred_Exp)))**2))
mapa_Exp = mean_absolute_percentage_error(test['sales'], pd.DataFrame(np.exp(pred_Exp)))

print('mapa : ', mapa_Exp)
print('rmse : ', rmse_Exp)

In [ ]:
#quadratic model

Quad = api.ols('sales ~ t + tsquare', data = train).fit()
pred_Quad = pd.Series(Quad.predict(test[["t", "tsquare"]]))
rmse_Quad = np.sqrt(np.mean((np.array(test['sales']) - np.array(pred_Quad))**2))
mapa_Quad = mean_absolute_percentage_error(test['sales'], pd.DataFrame(pred_Quad))

print('mapa : ', mapa_Quad)
print('rmse : ', rmse_Quad)


In [ ]:
# Additive Seasonality

add_sea = api.ols('sales ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov', data=train).fit()
pred_add_sea = pd.Series(add_sea.predict(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_add_sea = np.sqrt(np.mean((np.array(test['sales']) - np.array(pred_add_sea))**2))
mapa_add_sea = mean_absolute_percentage_error(test['sales'], pd.DataFrame(pred_add_sea))

print('mapa : ', mapa_add_sea)
print('rmse : ', rmse_add_sea)


In [ ]:
# Multiplicative Model

Mul_sea = api.ols('log_cement ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train).fit()
pred_Mult_sea = pd.Series(Mul_sea.predict(test))
rmse_Mult_sea = np.sqrt(np.mean((np.array(test['sales']) - np.array(np.exp(pred_Mult_sea)))**2))
mapa_Mult_sea = mean_absolute_percentage_error(test['sales'], pd.DataFrame(np.exp(pred_Mult_sea)))

print('mapa : ', mapa_Mult_sea)
print('rmse : ', rmse_Mult_sea)


In [ ]:
# Additive seasionality quadratic trend

add_sea_Quad = api.ols('sales ~ t+tsquare+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_add_sea_quad = pd.Series(add_sea_Quad.predict(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','tsquare']]))
rmse_add_sea_quad = np.sqrt(np.mean((np.array(test['sales'])-np.array(pred_add_sea_quad))**2))
mapa_add_sea_quad = mean_absolute_percentage_error(test['sales'], pd.DataFrame(pred_add_sea_quad))

print('mapa : ', mapa_add_sea_quad)
print('rmse : ', rmse_add_sea_quad)


In [ ]:
# Multiplicative seasonality linear trend

Mul_Add_sea = api.ols('log_cement ~ t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train).fit()
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(test))
rmse_Mult_add_sea = np.sqrt(np.mean((np.array(test['sales'])-np.array(np.exp(pred_Mult_add_sea)))**2))
mapa_Mult_add_sea = mean_absolute_percentage_error(test['sales'], pd.DataFrame(np.exp(pred_Mult_add_sea)))

print('mapa : ', mapa_Mult_add_sea)
print('rmse : ', rmse_Mult_add_sea )


In [ ]:
# concating data of model, rmse, mapa values so as to select appropriate model for better accuracy

data = {"MODEL":pd.Series(["rmse_linear","rmse_Exp","rmse_Quad","rmse_add_sea","rmse_add_sea_quad","rmse_Mult_sea","rmse_Mult_add_sea"]),
        "RMSE_Values":pd.Series([rmse_linear,rmse_Exp,rmse_Quad,rmse_add_sea,rmse_add_sea_quad,rmse_Mult_sea,rmse_Mult_add_sea]),
       "MAPA_Values": pd.Series([mapa_linear,mapa_Exp,mapa_Quad,mapa_add_sea,mapa_add_sea_quad,mapa_Mult_sea,mapa_Mult_add_sea])}
table_rmse_mapa = pd.DataFrame(data)
table_rmse_mapa

from above table we can say that,
RMSE_values, MAPA_values for Multiplicative seasonality linear trend model giving us less error,
so we can use this model for our prediction

In [ ]:
predict_data = pd.read_csv('/kaggle/input/forecasted-data-89/predictive data project 89.csv')
predict_data.head()

In [ ]:
p1= predict_data["month"][0]
p1[0:3]


In [ ]:
predict_data['months']= 0

for i in range(12):
    p1 = predict_data["month"][i]
    predict_data['months'][i]= p1[0:3]

predict_data

In [ ]:
predict_dummies = pd.DataFrame(pd.get_dummies(predict_data['months']))
predict_dummies.head()

In [ ]:
predict_data = pd.concat([predict_data, predict_dummies], axis= 1)
predict_data.head()

In [ ]:
# Final Model

model_full = api.ols('log_cement ~ t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = cementsales1).fit()


In [ ]:
pd.Series(model_full)

In [ ]:
pred_new = pd.Series(model_full.predict(predict_data))
pred_new

In [ ]:
pred_new_exp = np.exp(pred_new)
pred_new_exp

In [ ]:
# Forecasted values

predict_data['prediction_values'] = pred_new_exp
predict_data

# ARIMA model

auto_arima() function will be used to automatically select the best parameters for an ARIMA model. It takes several parameters to set a range of values for p, d, q, P, D, Q that the function will explore. For example, start_p=1, start_q=1 and max_p=3, max_q=3 are set as the range for p and q.


The auto_arima() model will use the stepwise=True option to fit the model iteratively and improve the model at each step.


The fitted model is then stored in the model_fit variable and the summary of the model is printed.


Finally, the code uses the predict() function of the fitted model to forecast the next 'n' periods of the time series.

In [ ]:
#!pip install pmdarima

In [ ]:
#importing auto arima so as to get values of (p, d, q) for Arima model

from pmdarima import auto_arima

In [ ]:
# Run combinations of ARIMA(p,d,q)
model_fit = auto_arima(train['sales'],
                       m=12,
                       d=0,
                       D=0,
                       max_order=None,
                       max_p=7,
                       max_q=7,
                       max_d=2,
                       max_P=4,
                       max_Q=4,
                       max_D=2,
                       maxiter = 50,
                       alpha = 0.05,
                       n_jobs = -1,
                       seasonal=True,
                       trace=True,
                       error_action='ignore',
                       suppress_warnings=True,
                       stepwise=True
                      )
model_fit.summary()

In [ ]:
model_fit

using best model parameter that got from auto-arima

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the ARIMA model

model_ARIMA = ARIMA(train['sales'],
              order=(2,0,0),
              seasonal_order=(1, 0, 2, 12))

In [ ]:
# Fit the model
model_ARIMA = model_ARIMA.fit()

train_forecast = train.copy()
test_forecast = test.copy()

train_forecast['forecast_ARIMA'] = model_ARIMA.predict()
train_forecast[['sales','forecast_ARIMA']].plot(figsize=(20,8))

In [ ]:
#checking RMSE, MAE, MAPA values for train data

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(train_forecast['sales'], train_forecast['forecast_ARIMA'])))
print("MAE of Auto ARIMA:", mean_absolute_error(train_forecast['sales'], train_forecast['forecast_ARIMA']))
print("MAPA of Auto ARIMA:", mean_absolute_percentage_error(train_forecast['sales'], train_forecast['forecast_ARIMA']))

#forcasting on test data

In [ ]:
# Forecast and compare against test data

# start date
start = len(train)

# End date
end = len(train)+len(test)-1

test_forecast['forecast_ARIMA'] = model_ARIMA.predict(start=start, end=end)
test_forecast[['sales','forecast_ARIMA']].plot(figsize=(20,8))

In [ ]:
#checking RMSE, MAE, MAPA values for test data

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(test_forecast['sales'], test_forecast['forecast_ARIMA'])))
print("MAE of Auto ARIMA:", mean_absolute_error(test_forecast['sales'], test_forecast['forecast_ARIMA']))
print("MAPE of Auto ARIMA:", mean_absolute_percentage_error(test_forecast['sales'], test_forecast['forecast_ARIMA']))

# Forcasting for Demand

# Linear Model

In [ ]:
# breaking up a time series into components, most notably: a trend component, a seasonal component, and a residual component

from statsmodels.tsa.seasonal import seasonal_decompose

decompose_ts_add1 = seasonal_decompose(cement['demand'], model = "additive", period = 12)
print(decompose_ts_add1.trend)
print(decompose_ts_add1.seasonal)
print(decompose_ts_add1.resid)
print(decompose_ts_add1.observed)
decompose_ts_add1.plot()

In [ ]:
decompose_ts_mul1 = seasonal_decompose(cement['demand'], model = "multiplicative", period = 12)
decompose_ts_mul1.plot()

In [ ]:
# filtering requied features for sales forecasting

cementdemand = cement.loc[:, ['Month','demand', 't']]
cementdemand.head()

In [ ]:
# squaring t value as it will further requied for model

cementdemand['tsquare'] = cementdemand['t'] * cementdemand['t']
cementdemand.head()

In [ ]:
# taking the log of sales values as it will be required for the exponential model

cementdemand['log_cement'] = np.log(cementdemand['demand'])

cementdemand.head()

In [ ]:
p3 = cementdemand["Month"][0]
p3[0:3]

cementdemand['months']= 0

for i in range(155):
    p3 = cementdemand["Month"][i]
    cementdemand['months'][i]= p3[0:3]

cementdemand

In [ ]:
# converting months catagorical features to numeric columns by using pandas get_dummies function

month_dummies1 = pd.DataFrame(pd.get_dummies(cementdemand['months']))
month_dummies1.head()

In [ ]:
# concating data

cementdemand1 = pd.concat([cementdemand, month_dummies1], axis= 1)
cementdemand1.head()

In [ ]:
cementdemand1.shape

In [ ]:
# splitting data into train & test

train1 = cementdemand1.head(143)
test1 = cementdemand1.tail(12)

In [ ]:
print(train1.shape)
print(test1.shape)

In [ ]:
train1.head()

In [ ]:
test1.tail()

In [ ]:
#importing linear Model, error matrix for error calculation
#checking different models for better accuracy

import statsmodels.formula.api as api
from sklearn.metrics import mean_absolute_percentage_error

#Linear Model

linear_model1 = api.ols('demand ~ t', data = train1).fit()
pred_linear1 =  pd.Series(linear_model1.predict(pd.DataFrame(test1['t'])))
rmse_linear1 = np.sqrt(np.mean((np.array(test1['demand']) - np.array(pred_linear1))**2))
mapa_linear1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(pred_linear1))

print('mapa : ', mapa_linear1)
print('rmse : ',rmse_linear1)

In [ ]:
# Exponential Model

Exp1 = api.ols('log_cement ~ t', data = train1).fit()
pred_Exp1 = pd.Series(Exp1.predict(pd.DataFrame(test1['t'])))
rmse_Exp1 = np.sqrt(np.mean((np.array(test1['demand']) - np.array(np.exp(pred_Exp1)))**2))
mapa_Exp1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(np.exp(pred_Exp1)))

print('mapa : ', mapa_Exp1)
print('rmse : ', rmse_Exp1)

In [ ]:
#quadratic model

Quad1 = api.ols('demand ~ t + tsquare', data = train1).fit()
pred_Quad1 = pd.Series(Quad1.predict(test1[["t", "tsquare"]]))
rmse_Quad1 = np.sqrt(np.mean((np.array(test1['demand']) - np.array(pred_Quad1))**2))
mapa_Quad1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(pred_Quad1))

print('mapa : ', mapa_Quad1)
print('rmse : ', rmse_Quad1)


In [ ]:
# Additive Seasonality

add_sea1 = api.ols('demand ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov', data=train1).fit()
pred_add_sea1 = pd.Series(add_sea1.predict(test1[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_add_sea1 = np.sqrt(np.mean((np.array(test1['demand']) - np.array(pred_add_sea1))**2))
mapa_add_sea1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(pred_add_sea1))

print('mapa : ', mapa_add_sea1)
print('rmse : ', rmse_add_sea1)


In [ ]:
# Multiplicative Model

Mul_sea1 = api.ols('log_cement ~ Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train1).fit()
pred_Mult_sea1 = pd.Series(Mul_sea1.predict(test1))
rmse_Mult_sea1 = np.sqrt(np.mean((np.array(test1['demand']) - np.array(np.exp(pred_Mult_sea1)))**2))
mapa_Mult_sea1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(np.exp(pred_Mult_sea1)))

print('mapa : ', mapa_Mult_sea1)
print('rmse : ', rmse_Mult_sea1)


In [ ]:
# Additive seasionality quadratic trend

add_sea_Quad1 = api.ols('demand ~ t+tsquare+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train1).fit()
pred_add_sea_quad1 = pd.Series(add_sea_Quad1.predict(test1[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','tsquare']]))
rmse_add_sea_quad1 = np.sqrt(np.mean((np.array(test1['demand'])-np.array(pred_add_sea_quad1))**2))
mapa_add_sea_quad1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(pred_add_sea_quad1))

print('mapa : ', mapa_add_sea_quad1)
print('rmse : ', rmse_add_sea_quad1)

In [ ]:
# Multiplicative seasonality linear trend

Mul_Add_sea1 = api.ols('log_cement ~ t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train1).fit()
pred_Mult_add_sea1 = pd.Series(Mul_Add_sea1.predict(test1))
rmse_Mult_add_sea1 = np.sqrt(np.mean((np.array(test1['demand'])-np.array(np.exp(pred_Mult_add_sea1)))**2))
mapa_Mult_add_sea1 = mean_absolute_percentage_error(test1['demand'], pd.DataFrame(np.exp(pred_Mult_add_sea1)))

print('mapa : ', mapa_Mult_add_sea1)
print('rmse : ', rmse_Mult_add_sea1)

In [ ]:
# concating data of model, rmse, mapa values so as to select appropriate model for better accuracy

data1 = {"MODEL":pd.Series(["rmse_linear","rmse_Exp","rmse_Quad","rmse_add_sea","rmse_add_sea_quad","rmse_Mult_sea","rmse_Mult_add_sea"]),
        "RMSE_Values":pd.Series([rmse_linear,rmse_Exp1,rmse_Quad1,rmse_add_sea1,rmse_add_sea_quad1,rmse_Mult_sea1,rmse_Mult_add_sea1]),
       "MAPA_Values": pd.Series([mapa_linear1,mapa_Exp,mapa_Quad1,mapa_add_sea1,mapa_add_sea_quad1,mapa_Mult_sea1,mapa_Mult_add_sea1])}
table_rmse_mapa1 = pd.DataFrame(data1)
table_rmse_mapa1

from above table we can say that, RMSE_values, MAPA_values for Multiplicative seasonality linear trend model giving us less error, so we can use this model for our prediction

# ARIMA Model

In [ ]:
#importing auto arima so as to get values of (p, d, q) for Arima model

from pmdarima import auto_arima


In [ ]:
# Run combinations of ARIMA(p,d,q)
model_fit1 = auto_arima(train1['demand'],
                       m=12,
                       d=0,
                       D=0,
                       max_order=None,
                       max_p=7,
                       max_q=7,
                       max_d=2,
                       max_P=4,
                       max_Q=4,
                       max_D=2,
                       maxiter = 50,
                       alpha = 0.05,
                       n_jobs = -1,
                       seasonal=True,
                       trace=True,
                       error_action='ignore',
                       suppress_warnings=True,
                       stepwise=True
                      )
model_fit1.summary()

In [ ]:
model_fit1

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the ARIMA model

model_ARIMA1 = ARIMA(train1['demand'],
              order=(2,0,0),
              seasonal_order=(1, 0, 1, 12))

In [ ]:
# Fit the model
model_ARIMA1 = model_ARIMA1.fit()

train_forecast1 = train1.copy()
test_forecast1 = test1.copy()

train_forecast1['forecast_ARIMA1'] = model_ARIMA1.predict()
train_forecast1[['demand','forecast_ARIMA1']].plot(figsize=(20,8))

In [ ]:
#checking RMSE, MAE, MAPA values for train data

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(train_forecast1['demand'], train_forecast1['forecast_ARIMA1'])))
print("MAE of Auto ARIMA:", mean_absolute_error(train_forecast1['demand'], train_forecast1['forecast_ARIMA1']))
print("MAPA of Auto ARIMA:", mean_absolute_percentage_error(train_forecast1['demand'], train_forecast1['forecast_ARIMA1']))

#forcasting on test data

In [ ]:
# Forecast and compare against test data

# start date
start1 = len(train1)

# End date
end1 = len(train1)+len(test1)-1

test_forecast1['forecast_ARIMA1'] = model_ARIMA1.predict(start=start1, end=end1)
test_forecast1[['demand','forecast_ARIMA1']].plot(figsize=(20,8))


In [ ]:
#checking RMSE, MAE, MAPA values for test data

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(test_forecast1['demand'], test_forecast1['forecast_ARIMA1'])))
print("MAE of Auto ARIMA:", mean_absolute_error(test_forecast1['demand'], test_forecast1['forecast_ARIMA1']))
print("MAPE of Auto ARIMA:", mean_absolute_percentage_error(test_forecast1['demand'], test_forecast1['forecast_ARIMA1']))